# PP2A - Single gene analysis in GP2 data

## Description

Using "individual level data"

### 0. Getting Started

- Loading Python libraries
- Defining functions
- Installing packages

### 1. Copy data from workspace to cloud environment

### 2. Extract PP2A

### 3. Annotate PP2A variants

### 4. Extract coding/non-syn variants

### 5. Calculate frequency in cases versus controls

### 6. Calculate frequency (homozygotes) in cases versus controls

### 7. Save out results

## Getting Started

### Loading Python libraries

### Defining functions

In [2]:
# Use the os package to interact with the environment
import os

# Bring in Pandas for Dataframe functionality
import pandas as pd

# Numpy for basics
import numpy as np

# Use StringIO for working with file contents
from io import StringIO

# Enable IPython to display matplotlib graphs
import matplotlib.pyplot as plt
%matplotlib inline

# Enable interaction with the FireCloud API
from firecloud import api as fapi

# Import the iPython HTML rendering for displaying links to Google Cloud Console
from IPython.core.display import display, HTML

# Import urllib modules for building URLs to Google Cloud Console
import urllib.parse

# BigQuery for querying data
from google.cloud import bigquery

#Import Sys
import sys as sys

In [3]:
# Utility routine for printing a shell command before executing it
def shell_do(command):
    print(f'Executing: {command}', file=sys.stderr)
    !$command
    
def shell_return(command):
    print(f'Executing: {command}', file=sys.stderr)
    output = !$command
    return '\n'.join(output)

# Utility routine for printing a query before executing it
def bq_query(query):
    print(f'Executing: {query}', file=sys.stderr)
    return pd.read_gbq(query, project_id=BILLING_PROJECT_ID, dialect='standard')

# Utility routine for display a message and a link
def display_html_link(description, link_text, url):
    html = f'''
    <p>
    </p>
    <p>
    {description}
    <a target=_blank href="{url}">{link_text}</a>.
    </p>
    '''

    display(HTML(html))

# Utility routines for reading files from Google Cloud Storage
def gcs_read_file(path):
    """Return the contents of a file in GCS"""
    contents = !gsutil -u {BILLING_PROJECT_ID} cat {path}
    return '\n'.join(contents)
    
def gcs_read_csv(path, sep=None):
    """Return a DataFrame from the contents of a delimited file in GCS"""
    return pd.read_csv(StringIO(gcs_read_file(path)), sep=sep, engine='python')

# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_gcs(description, link_text, gcs_path):
    url = '{}?{}'.format(
        os.path.join('https://console.cloud.google.com/storage/browser',
                     gcs_path.replace("gs://","")),
        urllib.parse.urlencode({'userProject': BILLING_PROJECT_ID}))

    display_html_link(description, link_text, url)

### Set paths

In [4]:
# Set up billing project and data path variables
BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']
WORKSPACE_NAMESPACE = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE_NAME = os.environ['WORKSPACE_NAME']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']

WORKSPACE_ATTRIBUTES = fapi.get_workspace(WORKSPACE_NAMESPACE, WORKSPACE_NAME).json().get('workspace',{}).get('attributes',{})

## Print the information to check we are in the proper release and billing 
## This will be different for you, the user, depending on the billing project your workspace is on
print('Billing and Workspace')
print(f'Workspace Name: {WORKSPACE_NAME}')
print(f'Billing Project: {BILLING_PROJECT_ID}')
print(f'Workspace Bucket, where you can upload and download data: {WORKSPACE_BUCKET}')
print('')


## GP2 v3.0
## Explicitly define release v3.0 path 
GP2_RELEASE_PATH = 'gs://gp2tier2/release3_31102022'
GP2_CLINICAL_RELEASE_PATH = f'{GP2_RELEASE_PATH}/clinical_data'
GP2_RAW_GENO_PATH = f'{GP2_RELEASE_PATH}/raw_genotypes'
GP2_IMPUTED_GENO_PATH = f'{GP2_RELEASE_PATH}/imputed_genotypes'
print('GP2 v3.0')
print(f'Path to GP2 v3.0 Clinical Data: {GP2_CLINICAL_RELEASE_PATH}')
print(f'Path to GP2 v3.0 Raw Genotype Data: {GP2_RAW_GENO_PATH}')
print(f'Path to GP2 v3.0 Imputed Genotype Data: {GP2_IMPUTED_GENO_PATH}')


## AMP-PD v3.0
## Explicitly define release v3.0 path 
AMP_RELEASE_PATH = 'gs://amp-pd-data/releases/2022_v3release_1115'
AMP_CLINICAL_RELEASE_PATH = f'{AMP_RELEASE_PATH}/clinical'

AMP_WGS_RELEASE_PATH = 'gs://amp-pd-genomics/releases/2022_v3release_1115/wgs-WB-DWGS'
AMP_WGS_RELEASE_PLINK_PATH = os.path.join(AMP_WGS_RELEASE_PATH, 'plink')
AMP_WGS_RELEASE_PLINK_PFILES = os.path.join(AMP_WGS_RELEASE_PLINK_PATH, 'pfiles')

print('AMP-PD v3.0')
print(f'Path to AMP-PD v3.0 Clinical Data: {AMP_CLINICAL_RELEASE_PATH}')
print(f'Path to AMP-PD v3.0 WGS Data: {AMP_WGS_RELEASE_PLINK_PATH}')
print(f'Path to AMP-PD v3.0 WGS Data: {AMP_WGS_RELEASE_PLINK_PFILES}')
print('')

Billing and Workspace
Workspace Name: PP2A
Billing Project: terra-02e7cd23
Workspace Bucket, where you can upload and download data: gs://fc-a2382a6f-59cf-4cf3-b70e-8cd83806d5e0

GP2 v3.0
Path to GP2 v3.0 Clinical Data: gs://gp2tier2/release3_31102022/clinical_data
Path to GP2 v3.0 Raw Genotype Data: gs://gp2tier2/release3_31102022/raw_genotypes
Path to GP2 v3.0 Imputed Genotype Data: gs://gp2tier2/release3_31102022/imputed_genotypes
AMP-PD v3.0
Path to AMP-PD v3.0 Clinical Data: gs://amp-pd-data/releases/2022_v3release_1115/clinical
Path to AMP-PD v3.0 WGS Data: gs://amp-pd-genomics/releases/2022_v3release_1115/wgs-WB-DWGS/plink
Path to AMP-PD v3.0 WGS Data: gs://amp-pd-genomics/releases/2022_v3release_1115/wgs-WB-DWGS/plink/pfiles



### Install packages

#### Install Plink 1.9 and Plink 2.0

In [5]:
%%bash

mkdir -p ~/tools
cd ~/tools

if test -e /home/jupyter/tools/plink; then
echo "Plink1.9 is already installed in /home/jupyter/tools/"

else
echo -e "Downloading plink \n    -------"
wget -N http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20190304.zip 
unzip -o plink_linux_x86_64_20190304.zip
echo -e "\n plink downloaded and unzipped in /home/jupyter/tools \n "

fi


if test -e /home/jupyter/tools/plink2; then
echo "Plink2 is already installed in /home/jupyter/tools/"

else
echo -e "Downloading plink2 \n    -------"
wget -N https://s3.amazonaws.com/plink2-assets/alpha3/plink2_linux_avx2_20220603.zip
unzip -o plink2_linux_avx2_20220603.zip
echo -e "\n plink2 downloaded and unzipped in /home/jupyter/tools \n "

fi

Plink1.9 is already installed in /home/jupyter/tools/
Plink2 is already installed in /home/jupyter/tools/


In [6]:
%%bash
ls /home/jupyter/tools/

annovar
annovar.latest.tar.gz
LICENSE
plink
plink2
plink2_linux_avx2_20220603.zip
plink_linux_x86_64_20190304.zip
prettify
toy.map
toy.ped


#### Remote restrictions

In [7]:
%%bash

# chmod plink 1.9 
chmod u+x /home/jupyter/tools/plink

In [8]:
%%bash

# chmod plink 2.0
chmod u+x /home/jupyter/tools/plink2

### Install ANNOVAR

In [9]:
%%bash

# Install ANNOVAR:
# https://www.openbioinformatics.org/annovar/annovar_download_form.php

if test -e /home/jupyter/tools/annovar; then

echo "annovar is already installed in /home/jupyter/tools/"
else
echo "annovar is not installed"
cd /home/jupyter/tools/

wget http://www.openbioinformatics.org/annovar/download/0wgxR2rIVP/annovar.latest.tar.gz

tar xvfz annovar.latest.tar.gz

fi

annovar is already installed in /home/jupyter/tools/


In [10]:
%%bash
ls /home/jupyter/tools/

annovar
annovar.latest.tar.gz
LICENSE
plink
plink2
plink2_linux_avx2_20220603.zip
plink_linux_x86_64_20190304.zip
prettify
toy.map
toy.ped


#### Install ANNOVAR: Download sources of annotation 

In [11]:
%%bash

cd /home/jupyter/tools/annovar/

perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar refGene humandb/
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar clinvar_20140902 humandb/
#perl annotate_variation.pl -buildver hg38 -downdb cytoBand humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ensGene humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar exac03 humandb/ 
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar avsnp147 humandb/ 
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar dbnsfp30a humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar gnomad211_genome humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ljb26_all humandb/

NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGene.txt.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGeneMrna.fa.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGeneVersion.txt.gz ... OK
NOTICE: Uncompressing downloaded files
NOTICE: Finished downloading annotation files for hg38 build version, with files saved at the 'humandb' directory
NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_clinvar_20140902.txt.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_clinvar_20140902.txt.idx.gz ... OK
NOTICE: Uncompressing downloaded files
NOTICE: Finished d

In [12]:
%%bash
ls /home/jupyter/tools/annovar/

annotate_variation.pl
coding_change.pl
convert2annovar.pl
example
humandb
retrieve_seq_from_fasta.pl
table_annovar.pl
variants_reduction.pl


## AJ

## Copy data from GP2 bucket to workspace

In [13]:
# Make a directory
print("Making a working directory")
WORK_DIR = f'/home/jupyter/PP2A_GP2/'
shell_do(f'mkdir -p {WORK_DIR}') # f' significa f-string - contiene expresiones para ejecutar el codigo

Making a working directory


Executing: mkdir -p /home/jupyter/PP2A_GP2/


In [14]:
# Check directory where AMP-PD data is
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GP2_IMPUTED_GENO_PATH}')

Executing: gsutil -u terra-02e7cd23 ls gs://gp2tier2/release3_31102022/imputed_genotypes


gs://gp2tier2/release3_31102022/imputed_genotypes/AAC/
gs://gp2tier2/release3_31102022/imputed_genotypes/AFR/
gs://gp2tier2/release3_31102022/imputed_genotypes/AJ/
gs://gp2tier2/release3_31102022/imputed_genotypes/AMR/
gs://gp2tier2/release3_31102022/imputed_genotypes/CAS/
gs://gp2tier2/release3_31102022/imputed_genotypes/EAS/
gs://gp2tier2/release3_31102022/imputed_genotypes/EUR/
gs://gp2tier2/release3_31102022/imputed_genotypes/MDE/
gs://gp2tier2/release3_31102022/imputed_genotypes/SAS/


In [15]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {GP2_IMPUTED_GENO_PATH}/AJ/chr9* {WORK_DIR}')

Executing: gsutil -u terra-02e7cd23 -m cp -r gs://gp2tier2/release3_31102022/imputed_genotypes/AJ/chr9* /home/jupyter/PP2A_GP2/


Copying gs://gp2tier2/release3_31102022/imputed_genotypes/AJ/chr9_AJ_release3.psam...
Copying gs://gp2tier2/release3_31102022/imputed_genotypes/AJ/chr9_AJ_release3.pgen...
Copying gs://gp2tier2/release3_31102022/imputed_genotypes/AJ/chr9_AJ_release3.pvar...
Copying gs://gp2tier2/release3_31102022/imputed_genotypes/AJ/chr9_AJ_release3.log...
| [4/4 files][300.0 MiB/300.0 MiB] 100% Done                                    
Operation completed over 4 objects/300.0 MiB.                                    


### Create a covariate file with GP2 data

In [16]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GP2_CLINICAL_RELEASE_PATH}')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {GP2_CLINICAL_RELEASE_PATH}/master_key_release3_final.csv {WORK_DIR}')

Executing: gsutil -u terra-02e7cd23 ls gs://gp2tier2/release3_31102022/clinical_data


gs://gp2tier2/release3_31102022/clinical_data/
gs://gp2tier2/release3_31102022/clinical_data/master_key_release3_final.csv
gs://gp2tier2/release3_31102022/clinical_data/release3_31102022_data_dictionary.csv


Executing: gsutil -u terra-02e7cd23 -m cp -r gs://gp2tier2/release3_31102022/clinical_data/master_key_release3_final.csv /home/jupyter/PP2A_GP2/


Copying gs://gp2tier2/release3_31102022/clinical_data/master_key_release3_final.csv...
/ [1/1 files][  1.6 MiB/  1.6 MiB] 100% Done                                    
Operation completed over 1 objects/1.6 MiB.                                      


In [17]:
cov = pd.read_csv(f'{WORK_DIR}/master_key_release3_final.csv')
cov.columns

Index(['GP2ID', 'GP2sampleID', 'manifest_id', 'phenotype', 'pheno_for_qc',
       'other_pheno', 'sex_for_qc', 'age', 'age_of_onset', 'age_at_diagnosis',
       'age_at_death', 'race_for_qc', 'family_history_for_qc', 'region_for_qc',
       'study', 'pruned', 'pruned_reason', 'label', 'related'],
      dtype='object')

In [18]:
cov_reduced = cov[['GP2sampleID','GP2sampleID','sex_for_qc', 'age', 'phenotype']]
cov_reduced.columns = ['FID','IID', 'SEX','AGE','PHENO']
cov_reduced

,FID,IID,SEX,AGE,PHENO
0,SYNAPS-KZ_000001_s1,SYNAPS-KZ_000001_s1,2,64.0,Control
1,SYNAPS-KZ_000002_s1,SYNAPS-KZ_000002_s1,1,53.0,Control
2,SYNAPS-KZ_000003_s1,SYNAPS-KZ_000003_s1,2,59.0,Control
3,SYNAPS-KZ_000004_s1,SYNAPS-KZ_000004_s1,1,57.0,Control
4,SYNAPS-KZ_000005_s1,SYNAPS-KZ_000005_s1,1,59.0,Control
...,...,...,...,...,...
16852,PPMI_000789_s1,PPMI_000789_s1,2,NaN,Other
16853,PPMI_000781_s1,PPMI_000781_s1,1,57.0,Other
16854,PPMI_000818_s1,PPMI_000818_s1,1,60.0,Other
16855,PPMI_000841_s1,PPMI_000841_s1,2,68.0,Other


In [19]:
conditions = [
    (cov_reduced['PHENO'] == "Case"),
    (cov_reduced['PHENO'] == "Control")]

In [20]:
choices = [2,1]
cov_reduced['PHENOTYPE'] = np.select(conditions, choices, default=-9).astype(np.int64)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [21]:
cov_reduced.reset_index(inplace=True)
cov_reduced.drop(columns=["index"], inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [22]:
cov_reduced.drop(columns=['PHENO'], inplace=True)

In [23]:
sex = cov_reduced[['FID','IID','SEX']]
sex.to_csv(f'{WORK_DIR}/SEX.txt',index=False, sep="\t")

In [24]:
pheno = cov_reduced[['FID','IID','PHENOTYPE']]
pheno.to_csv(f'{WORK_DIR}/PHENO.txt',index=False, sep="\t")

## Extract PP2A

In [25]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

# PODXL: gene positions on hg38 (from https://useast.ensembl.org/index.html)
/home/jupyter/tools/plink2 \
--pfile chr9_AJ_release3 \
--chr 9 \
--from-bp 129110950  \
--to-bp 129148946 \
--make-bed \
--out pheno_PP2A_AJ

PLINK v2.00a3.3LM AVX2 Intel (3 Jun 2022)      www.cog-genomics.org/plink/2.0/
(C) 2005-2022 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to pheno_PP2A_AJ.log.
Options in effect:
  --chr 9
  --from-bp 129110950
  --make-bed
  --out pheno_PP2A_AJ
  --pfile chr9_AJ_release3
  --to-bp 129148946

Start time: Sun Jun  4 21:13:31 2023
14998 MiB RAM detected; reserving 7499 MiB for main workspace.
Using up to 4 compute threads.
1285 samples (477 females, 808 males; 1285 founders) loaded from
chr9_AJ_release3.psam.
524194 variants loaded from chr9_AJ_release3.pvar.
1 binary phenotype loaded (600 cases, 323 controls).
177 variants remaining after main filters.
Writing pheno_PP2A_AJ.fam ... done.
Writing pheno_PP2A_AJ.bim ... done.
Writing pheno_PP2A_AJ.bed ... 0%done.
End time: Sun Jun  4 21:13:31 2023


In [26]:
%%bash 
WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

grep -e "CORIELL_003933_s1" master_key_release3_final.csv > GP2_AJ_comphet.txt
head GP2_AJ_comphet.txt

CORIELL_003933,CORIELL_003933_s1,m3,PD,2,PD,2,78.0,60.0,,,White,Yes,USA,CORIELL,0,,AJ,


### Visualize plink files bim, fam and bed

In [27]:
%%bash

# Visualize bim file
WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

head pheno_PP2A_AJ.bim

9	chr9:129110977:A:C	0	129110977	C	A
9	chr9:129111323:C:G	0	129111323	G	C
9	chr9:129111384:C:A	0	129111384	A	C
9	chr9:129111646:GC:G	0	129111646	G	GC
9	chr9:129112273:A:T	0	129112273	T	A
9	chr9:129112362:A:G	0	129112362	G	A
9	chr9:129112974:T:C	0	129112974	C	T
9	chr9:129113056:T:TA	0	129113056	TA	T
9	chr9:129113634:GTGGCACACGTC:G	0	129113634	G	GTGGCACACGTC
9	chr9:129113698:A:T	0	129113698	T	A


In [28]:
%%bash

# Visualize fam file
WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

head pheno_PP2A_AJ.fam

0	APGS_000021_s1	0	0	1	2
0	APGS_000475_s1	0	0	1	2
0	APGS_000486_s1	0	0	1	2
0	APGS_000646_s1	0	0	2	2
0	APGS_000808_s1	0	0	1	2
0	APGS_001017_s1	0	0	1	2
0	APGS_001059_s1	0	0	1	2
0	APGS_001066_s1	0	0	2	2
0	APGS_001426_s1	0	0	2	2
0	APGS_001474_s1	0	0	2	2


In [29]:
%%bash

# Visualize fam file
WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

head pheno_PP2A_AJ.bed

l�������>�����:/��+��������ꫮ����/�;�����������+���������<��������+����������ꪫ������ί������������ξ����������Ͽ������ﾸ쏺�����������������?�ﺿ�������:�����������?�����Ϻ����������.�����������������������������>����?���������?���+��������.>��������?꿻�������8�Ϊ꾿�����ʈ��������캯������ﺪ����������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������

In [30]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

# Turn binary files into VCF
/home/jupyter/tools/plink \
--bfile pheno_PP2A_AJ \
--recode vcf-fid \
--out pheno_PP2A_AJ

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to pheno_PP2A_AJ.log.
Options in effect:
  --bfile pheno_PP2A_AJ
  --out pheno_PP2A_AJ
  --recode vcf-fid

14998 MB RAM detected; reserving 7499 MB for main workspace.
177 variants loaded from .bim file.
1285 people (808 males, 477 females) loaded from .fam.
923 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 1285 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%38%39%40%41%42%43%44%45%46%47%48%49%50%51%52%53%54%55%56%57%58%59%60%61%62%63%64%65%

## Annotate PP2A variants

In [31]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR
export PATH=$PATH:/home/jupyter/tools/rvtests/third/tabix-0.2.6/

### Bgzip and Tabix
bgzip pheno_PP2A_AJ.vcf

tabix -f -p vcf pheno_PP2A_AJ.vcf.gz
tabix -f -p vcf pheno_PP2A_AJ.vcf.gz

[bgzip] can't create pheno_PP2A_AJ.vcf.gz: File exists


In [32]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

### Annotate variants using ANNOVAR: https://annovar.openbioinformatics.org/en/latest/ 
perl /home/jupyter/tools/annovar/table_annovar.pl pheno_PP2A_AJ.vcf.gz /home/jupyter/tools/annovar/humandb/ -buildver hg38 \
-out pheno_PP2A_AJ.annovar \
-remove -protocol refGene,clinvar_20140902 \
-operation g,f \
--nopolish \
-nastring . \
-vcfinput


NOTICE: Running with system command <convert2annovar.pl  -includeinfo -allsample -withfreq -format vcf4 pheno_PP2A_AJ.vcf.gz > pheno_PP2A_AJ.annovar.avinput>
NOTICE: Finished reading 184 lines from VCF file
NOTICE: A total of 177 locus in VCF file passed QC threshold, representing 161 SNPs (118 transitions and 43 transversions) and 16 indels/substitutions
NOTICE: Finished writing allele frequencies based on 206885 SNP genotypes (151630 transitions and 55255 transversions) and 20560 indels/substitutions for 1285 samples

NOTICE: Running with system command </home/jupyter/tools/annovar/table_annovar.pl pheno_PP2A_AJ.annovar.avinput /home/jupyter/tools/annovar/humandb/ -buildver hg38 -outfile pheno_PP2A_AJ.annovar -remove -protocol refGene,clinvar_20140902 -operation g,f --nopolish -nastring . -otherinfo>
-----------------------------------------------------------------
NOTICE: Processing operation=g protocol=refGene

NOTICE: Running with system command <annotate_variation.pl -geneanno -

## Extract coding and non-syn variants

In [33]:
# Visualize multianno file
PP2A_AJ = pd.read_csv(f'{WORK_DIR}/pheno_PP2A_AJ.annovar.hg38_multianno.txt', sep = '\t')
PP2A_AJ.head()

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo1288,Otherinfo1289,Otherinfo1290,Otherinfo1291,Otherinfo1292,Otherinfo1293,Otherinfo1294,Otherinfo1295,Otherinfo1296,Otherinfo1297
0,9,129110977,129110977,A,C,UTR5,PTPA,NM_021131:c.-624A>C,.,.,...,0/0,0/0,0/1,0/0,0/0,0/0,0/0,0/0,0/0,0/0
1,9,129111323,129111323,C,G,UTR5,PTPA,NM_178001:c.-278C>G;NM_001271832:c.-278C>G,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
2,9,129111384,129111384,C,A,UTR5,PTPA,NM_178001:c.-217C>A;NM_001271832:c.-217C>A,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
3,9,129111647,129111647,C,-,intronic,PTPA,.,.,.,...,0/0,0/1,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
4,9,129112273,129112273,T,A,intronic,PTPA,.,.,.,...,0/0,0/0,0/1,0/0,0/0,0/0,0/0,0/0,0/0,0/0


In [34]:
PP2A_AJ.count()

Chr              177
Start            177
End              177
Ref              177
Alt              177
                ... 
Otherinfo1293    177
Otherinfo1294    177
Otherinfo1295    177
Otherinfo1296    177
Otherinfo1297    177
Length: 1308, dtype: int64

In [35]:
# Filter exonic variants
coding_AJ = PP2A_AJ[PP2A_AJ['Func.refGene'] == 'exonic']
coding_AJ.count()
coding_AJ.head(5)

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo1288,Otherinfo1289,Otherinfo1290,Otherinfo1291,Otherinfo1292,Otherinfo1293,Otherinfo1294,Otherinfo1295,Otherinfo1296,Otherinfo1297
79,9,129129094,129129094,A,G,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon3:c.A239G:p.Y80C,PTPA:NM...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
144,9,129142498,129142498,C,T,exonic,PTPA,.,synonymous SNV,"PTPA:NM_001271832:exon8:c.C753T:p.A251A,PTPA:N...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
168,9,129147457,129147457,C,T,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon9:c.C878T:p.S293L,PTPA:N...",...,0/0,0/0,0/1,0/0,0/0,0/0,0/0,0/0,0/0,0/0


In [36]:
# Filter exonic and non-syn vars
coding_nonsynonymous_AJ = PP2A_AJ[(PP2A_AJ['Func.refGene'] == 'exonic') & (PP2A_AJ['ExonicFunc.refGene'] == 'nonsynonymous SNV')]
coding_nonsynonymous_AJ.count(1)

79     1308
168    1308
dtype: int64

In [37]:
coding_nonsynonymous_AJ.head(1)

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo1288,Otherinfo1289,Otherinfo1290,Otherinfo1291,Otherinfo1292,Otherinfo1293,Otherinfo1294,Otherinfo1295,Otherinfo1296,Otherinfo1297
79,9,129129094,129129094,A,G,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon3:c.A239G:p.Y80C,PTPA:NM...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0


In [38]:
print(coding_nonsynonymous_AJ)

     Chr      Start        End Ref Alt Func.refGene Gene.refGene  \
79     9  129129094  129129094   A   G       exonic         PTPA   
168    9  129147457  129147457   C   T       exonic         PTPA   

    GeneDetail.refGene ExonicFunc.refGene  \
79                   .  nonsynonymous SNV   
168                  .  nonsynonymous SNV   

                                      AAChange.refGene  ... Otherinfo1288  \
79   PTPA:NM_001271832:exon3:c.A239G:p.Y80C,PTPA:NM...  ...           0/0   
168  PTPA:NM_001271832:exon9:c.C878T:p.S293L,PTPA:N...  ...           0/0   

     Otherinfo1289 Otherinfo1290 Otherinfo1291  Otherinfo1292  Otherinfo1293  \
79             0/0           0/0           0/0            0/0            0/0   
168            0/0           0/1           0/0            0/0            0/0   

    Otherinfo1294 Otherinfo1295 Otherinfo1296 Otherinfo1297  
79            0/0           0/0           0/0           0/0  
168           0/0           0/0           0/0           0/0  


In [39]:
coding_nonsynonymous_AJ.to_csv(f'{WORK_DIR}/coding_nonsynonymous.txt', sep = '\t', index = False)
coding_nonsynonymous_AJ.head(9)

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo1288,Otherinfo1289,Otherinfo1290,Otherinfo1291,Otherinfo1292,Otherinfo1293,Otherinfo1294,Otherinfo1295,Otherinfo1296,Otherinfo1297
79,9,129129094,129129094,A,G,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon3:c.A239G:p.Y80C,PTPA:NM...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
168,9,129147457,129147457,C,T,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon9:c.C878T:p.S293L,PTPA:N...",...,0/0,0/0,0/1,0/0,0/0,0/0,0/0,0/0,0/0,0/0


In [40]:
# Filter intronic variants
intronic_AJ = PP2A_AJ[PP2A_AJ['Func.refGene'] == 'intronic']
intronic_AJ.count()

Chr              163
Start            163
End              163
Ref              163
Alt              163
                ... 
Otherinfo1293    163
Otherinfo1294    163
Otherinfo1295    163
Otherinfo1296    163
Otherinfo1297    163
Length: 1308, dtype: int64

In [41]:
# Filter UTR3 variants
UTR3_AJ = PP2A_AJ[PP2A_AJ['Func.refGene'] == ('UTR3')]
UTR3_AJ.count()

Chr              8
Start            8
End              8
Ref              8
Alt              8
                ..
Otherinfo1293    8
Otherinfo1294    8
Otherinfo1295    8
Otherinfo1296    8
Otherinfo1297    8
Length: 1308, dtype: int64

In [42]:
# Filter UTR5 variants
UTR5_AJ = PP2A_AJ[PP2A_AJ['Func.refGene'] == ('UTR5')]
UTR5_AJ.count()

Chr              3
Start            3
End              3
Ref              3
Alt              3
                ..
Otherinfo1293    3
Otherinfo1294    3
Otherinfo1295    3
Otherinfo1296    3
Otherinfo1297    3
Length: 1308, dtype: int64

In [43]:
# Calculate freq - cases vs controls

In [44]:
reduced_coding_nonsynonymous_AJ = coding_nonsynonymous_AJ[["Chr", "Start", "End", "Gene.refGene"]]
reduced_coding_nonsynonymous_AJ.to_csv(f'{WORK_DIR}/reduced_coding_nonsynonymous_AJ.txt', sep = '\t', index = False, header= False)

In [45]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

head reduced_coding_nonsynonymous_AJ.txt

9	129129094	129129094	PTPA
9	129147457	129147457	PTPA


In [46]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_PP2A_AJ --extract range reduced_coding_nonsynonymous_AJ.txt --assoc --out coding_nonsynonymous_pheno_PP2A_AJ --allow-no-sex --ci 0.95

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_PP2A_AJ.log.
Options in effect:
  --allow-no-sex
  --assoc
  --bfile pheno_PP2A_AJ
  --ci 0.95
  --extract range reduced_coding_nonsynonymous_AJ.txt
  --out coding_nonsynonymous_pheno_PP2A_AJ

14998 MB RAM detected; reserving 7499 MB for main workspace.
177 variants loaded from .bim file.
1285 people (808 males, 477 females) loaded from .fam.
923 phenotype values loaded from .fam.
--extract range: 175 variants excluded.
--extract range: 2 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 1285 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%

In [47]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_PP2A_AJ --extract range reduced_coding_nonsynonymous_AJ.txt --make-bed --max-maf 0.05 --out coding_nonsynonymous_AJ_rare_0.05 --allow-no-sex

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_AJ_rare_0.05.log.
Options in effect:
  --allow-no-sex
  --bfile pheno_PP2A_AJ
  --extract range reduced_coding_nonsynonymous_AJ.txt
  --make-bed
  --max-maf 0.05
  --out coding_nonsynonymous_AJ_rare_0.05

14998 MB RAM detected; reserving 7499 MB for main workspace.
177 variants loaded from .bim file.
1285 people (808 males, 477 females) loaded from .fam.
923 phenotype values loaded from .fam.
--extract range: 175 variants excluded.
--extract range: 2 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 1285 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%

In [48]:
PP2A_AJ_freq = pd.read_csv(f'{WORK_DIR}/coding_nonsynonymous_pheno_PP2A_AJ.assoc', delim_whitespace=True)
PP2A_AJ_freq.head(10)

,CHR,SNP,BP,A1,F_A,F_U,A2,CHISQ,P,OR,SE,L95,U95
0,9,chr9:129129094:A:G,129129094,G,0.0075,0.00000,A,4.8690,0.02735,NaN,NaN,NaN,NaN
1,9,chr9:129147457:C:T,129147457,T,0.0576,0.06677,C,0.6173,0.43200,0.8542,0.2007,0.5764,1.266


In [49]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_PP2A_AJ --extract range reduced_coding_nonsynonymous_AJ.txt --assoc --out coding_nonsynonymous_pheno_PP2A_AJ --allow-no-sex --adjust

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_PP2A_AJ.log.
Options in effect:
  --adjust
  --allow-no-sex
  --assoc
  --bfile pheno_PP2A_AJ
  --extract range reduced_coding_nonsynonymous_AJ.txt
  --out coding_nonsynonymous_pheno_PP2A_AJ

14998 MB RAM detected; reserving 7499 MB for main workspace.
177 variants loaded from .bim file.
1285 people (808 males, 477 females) loaded from .fam.
923 phenotype values loaded from .fam.
--extract range: 175 variants excluded.
--extract range: 2 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 1285 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%

In [50]:
PP2A_AJ_freq = pd.read_csv(f'{WORK_DIR}/coding_nonsynonymous_pheno_PP2A_AJ.assoc.adjusted', delim_whitespace=True)
PP2A_AJ_freq.head(10)

,CHR,SNP,UNADJ,GC,BONF,HOLM,SIDAK_SS,SIDAK_SD,FDR_BH,FDR_BY
0,9,chr9:129129094:A:G,0.02735,0.3683,0.0547,0.0547,0.05395,0.05395,0.0547,0.08204
1,9,chr9:129147457:C:T,0.43200,0.7487,0.8641,0.4320,0.67740,0.43200,0.4320,0.64810


## Calculate freq of HMZ in cases versus controls

In [51]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_PP2A_AJ --extract range reduced_coding_nonsynonymous_AJ.txt --recode A --out coding_nonsynonymous_pheno_PP2A_AJ

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_PP2A_AJ.log.
Options in effect:
  --bfile pheno_PP2A_AJ
  --extract range reduced_coding_nonsynonymous_AJ.txt
  --out coding_nonsynonymous_pheno_PP2A_AJ
  --recode A

14998 MB RAM detected; reserving 7499 MB for main workspace.
177 variants loaded from .bim file.
1285 people (808 males, 477 females) loaded from .fam.
923 phenotype values loaded from .fam.
--extract range: 175 variants excluded.
--extract range: 2 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 1285 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%

In [52]:
PP2A_AJ_freq = pd.read_csv(f'{WORK_DIR}/coding_nonsynonymous_pheno_PP2A_AJ.assoc', delim_whitespace=True)
PP2A_AJ_freq.head()

,CHR,SNP,BP,A1,F_A,F_U,A2,CHISQ,P,OR
0,9,chr9:129129094:A:G,129129094,G,0.0075,0.00000,A,4.8690,0.02735,NaN
1,9,chr9:129147457:C:T,129147457,T,0.0576,0.06677,C,0.6173,0.43200,0.8542


In [53]:
PP2A_AJ_recode = pd.read_csv(f'{WORK_DIR}/coding_nonsynonymous_pheno_PP2A_AJ.raw', delim_whitespace=True)
PP2A_AJ_recode.head()

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129129094:A:G_G,chr9:129147457:C:T_T
0,0,APGS_000021_s1,0,0,1,2,0,0.0
1,0,APGS_000475_s1,0,0,1,2,0,0.0
2,0,APGS_000486_s1,0,0,1,2,0,0.0
3,0,APGS_000646_s1,0,0,2,2,0,0.0
4,0,APGS_000808_s1,0,0,1,2,0,0.0


In [54]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile  pheno_PP2A_AJ --extract range reduced_coding_nonsynonymous_AJ.txt --recode A --out coding_nonsynonymous_pheno_PP2A_AJ

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_PP2A_AJ.log.
Options in effect:
  --bfile pheno_PP2A_AJ
  --extract range reduced_coding_nonsynonymous_AJ.txt
  --out coding_nonsynonymous_pheno_PP2A_AJ
  --recode A

14998 MB RAM detected; reserving 7499 MB for main workspace.
177 variants loaded from .bim file.
1285 people (808 males, 477 females) loaded from .fam.
923 phenotype values loaded from .fam.
--extract range: 175 variants excluded.
--extract range: 2 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 1285 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%

In [55]:
# Explore HMZs for chr9:129147457:C:T_T in cases
PP2A_hom_casos_129147457_AJ = PP2A_AJ_recode[(PP2A_AJ_recode['chr9:129147457:C:T_T'] == 2) & (PP2A_AJ_recode['PHENOTYPE'] == 2)]
PP2A_hom_casos_129147457_AJ.head()

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129129094:A:G_G,chr9:129147457:C:T_T
35,0,BIOFIND_000043_s1,0,0,1,2,0,2.0
176,0,CORIELL_001619_s1,0,0,2,2,0,2.0


In [56]:
PP2A_hom_casos_129147457_AJ.shape

(2, 8)

In [57]:
PP2A_hom_casos_129147457_AJ.to_csv('PP2A_hom_casos_129147457_AJ.csv')

In [58]:
# Explore HMZs for chr9:129147457:C:T_T in controls
PP2A_hom_controles_129147457_AJ = PP2A_AJ_recode[(PP2A_AJ_recode['chr9:129147457:C:T_T'] == 2) & (PP2A_AJ_recode['PHENOTYPE'] == 1)]
PP2A_hom_controles_129147457_AJ.head()

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129129094:A:G_G,chr9:129147457:C:T_T
530,0,CORIELL_005579_s1,0,0,1,1,0,2.0


In [59]:
PP2A_hom_controles_129147457_AJ.shape

(1, 8)

In [60]:
# Explore HMZs for chr9:129129094:A:G_G in cases
PP2A_hom_casos_129129094_AJ = PP2A_AJ_recode[(PP2A_AJ_recode['chr9:129129094:A:G_G'] == 2) & (PP2A_AJ_recode['PHENOTYPE'] == 2)]
PP2A_hom_casos_129129094_AJ.head()

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129129094:A:G_G,chr9:129147457:C:T_T


In [61]:
PP2A_hom_casos_129129094_AJ.shape

(0, 8)

In [62]:
# Explore HMZs for chr9:129129094:A:G_G in controls
PP2A_hom_controles_129129094_AJ = PP2A_AJ_recode[(PP2A_AJ_recode['chr9:129129094:A:G_G'] == 2) & (PP2A_AJ_recode['PHENOTYPE'] == 1)]
PP2A_hom_controles_129129094_AJ.head()

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129129094:A:G_G,chr9:129147457:C:T_T


In [63]:
PP2A_hom_controles_129129094_AJ.shape

(0, 8)

In [64]:
# Identifying individuals het for the alternative allele
het = PP2A_AJ_recode[(PP2A_AJ_recode['chr9:129147457:C:T_T'] == 1) | (PP2A_AJ_recode['chr9:129129094:A:G_G'] == 1)]
het = het.dropna()
print(het.shape)

(161, 8)


In [65]:
het.head (161)

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129129094:A:G_G,chr9:129147457:C:T_T
12,0,BCM_000051_s1,0,0,1,2,0,1.0
14,0,BCM_000158_s1,0,0,1,2,0,1.0
37,0,BIOFIND_000049_s1,0,0,1,2,0,1.0
43,0,BIOFIND_000116_s1,0,0,2,1,0,1.0
45,0,BIOFIND_000126_s1,0,0,1,1,0,1.0
...,...,...,...,...,...,...,...,...
1257,0,UMD_000252_s1,0,0,2,2,0,1.0
1267,0,UMD_000346_s1,0,0,1,2,0,1.0
1268,0,UMD_000364_s1,0,0,1,2,0,1.0
1273,0,UMD_000386_s1,0,0,2,1,0,1.0


In [66]:
het['compound'] = het['chr9:129147457:C:T_T'] + het['chr9:129129094:A:G_G']
het['compound'].value_counts()

1.0    158
2.0      3
Name: compound, dtype: int64

In [67]:
comp_het_info_cases = het[(het['PHENOTYPE'] == 2) & (het['compound'] == 2)]
print(comp_het_info_cases.shape)

(1, 9)


In [68]:
comp_het_info_control = het[(het['PHENOTYPE'] == 1) & (het['compound'] == 2)]
print(comp_het_info_control.shape)

(0, 9)


In [69]:
comp_het_info_cases.to_csv('/home/jupyter/PTPA/comp_het_info_cases.txt', sep = '\t', index=True)
comp_het_info_cases.head(54)

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129129094:A:G_G,chr9:129147457:C:T_T,compound
421,0,CORIELL_003933_s1,0,0,2,2,1,1.0,2.0


In [70]:
comp_het_info_control.to_csv('/home/jupyter/PTPA/comp_het_info_control.txt', sep = '\t', index=True)
comp_het_info_control.head(54)

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129129094:A:G_G,chr9:129147457:C:T_T,compound


In [71]:
# Explore HETs for chr9:129147457:C:T_T in cases
PP2A_het_casos_129147457_AJ = PP2A_AJ_recode[(PP2A_AJ_recode['chr9:129147457:C:T_T'] == 1) & (PP2A_AJ_recode['PHENOTYPE'] == 2)]
PP2A_het_casos_129147457_AJ

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129129094:A:G_G,chr9:129147457:C:T_T
12,0,BCM_000051_s1,0,0,1,2,0,1.0
14,0,BCM_000158_s1,0,0,1,2,0,1.0
37,0,BIOFIND_000049_s1,0,0,1,2,0,1.0
48,0,BIOFIND_000136_s1,0,0,2,2,0,1.0
56,0,BIOFIND_000205_s1,0,0,1,2,0,1.0
...,...,...,...,...,...,...,...,...
1241,0,UMD_000047_s1,0,0,1,2,0,1.0
1257,0,UMD_000252_s1,0,0,2,2,0,1.0
1267,0,UMD_000346_s1,0,0,1,2,0,1.0
1268,0,UMD_000364_s1,0,0,1,2,0,1.0


In [72]:
PP2A_het_casos_129147457_AJ.shape

(65, 8)

In [73]:
# Explore HETs for chr9:129147457:C:T_T in controles
PP2A_het_controles_129147457_AJ = PP2A_AJ_recode[(PP2A_AJ_recode['chr9:129147457:C:T_T'] == 1) & (PP2A_AJ_recode['PHENOTYPE'] == 1)]
PP2A_het_controles_129147457_AJ

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129129094:A:G_G,chr9:129147457:C:T_T
43,0,BIOFIND_000116_s1,0,0,2,1,0,1.0
45,0,BIOFIND_000126_s1,0,0,1,1,0,1.0
484,0,CORIELL_004515_s1,0,0,2,1,0,1.0
485,0,CORIELL_004521_s1,0,0,2,1,0,1.0
487,0,CORIELL_004628_s1,0,0,1,1,0,1.0
501,0,CORIELL_005008_s1,0,0,1,1,0,1.0
517,0,CORIELL_005259_s1,0,0,2,1,0,1.0
520,0,CORIELL_005327_s1,0,0,1,1,0,1.0
523,0,CORIELL_005425_s1,0,0,2,1,0,1.0
532,0,CORIELL_005730_s1,0,0,2,1,0,1.0


In [74]:
PP2A_het_controles_129147457_AJ.shape

(41, 8)

In [75]:
# Explore HETs for chr9:129129094:A:G_G in cases
PP2A_het_casos_129129094_AJ = PP2A_AJ_recode[(PP2A_AJ_recode['chr9:129129094:A:G_G'] == 1) & (PP2A_AJ_recode['PHENOTYPE'] == 2)]
PP2A_het_casos_129129094_AJ.head(600)

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129129094:A:G_G,chr9:129147457:C:T_T
60,0,CORIELL_000035_s1,0,0,2,2,1,0.0
67,0,CORIELL_000138_s1,0,0,1,2,1,0.0
137,0,CORIELL_001140_s1,0,0,1,2,1,0.0
224,0,CORIELL_001997_s1,0,0,1,2,1,0.0
301,0,CORIELL_002302_s1,0,0,1,2,1,0.0
421,0,CORIELL_003933_s1,0,0,2,2,1,1.0
437,0,CORIELL_004167_s1,0,0,2,2,1,0.0
718,0,PAGE_001019_s1,0,0,1,2,1,0.0
1252,0,UMD_000157_s1,0,0,1,2,1,0.0


In [76]:
PP2A_het_casos_129129094_AJ.shape

(9, 8)

In [100]:
# Explore HETs for chr9:129129094:A:G_G in controles
PP2A_het_controles_129129094_AJ = PP2A_AJ_recode[(PP2A_AJ_recode['chr9:129129094:A:G_G'] == 1) & (PP2A_AJ_recode['PHENOTYPE'] == 1)]
PP2A_het_controles_129129094_AJ

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129129094:A:G_G,chr9:129147457:C:T_T


In [78]:
PP2A_het_controles_129129094_AJ.shape

(0, 8)

In [79]:
!pip install rpy2
%load_ext rpy2.ipython

In [80]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [81]:
import rpy2
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [82]:
PP2A_het_casos_129147457_AJ.to_csv('PP2A_het_casos_129147457_AJ.csv')

In [83]:
temp = pd.read_csv('PP2A_het_casos_129147457_AJ.csv')
temp

,Unnamed: 0,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129129094:A:G_G,chr9:129147457:C:T_T
0,12,0,BCM_000051_s1,0,0,1,2,0,1.0
1,14,0,BCM_000158_s1,0,0,1,2,0,1.0
2,37,0,BIOFIND_000049_s1,0,0,1,2,0,1.0
3,48,0,BIOFIND_000136_s1,0,0,2,2,0,1.0
4,56,0,BIOFIND_000205_s1,0,0,1,2,0,1.0
...,...,...,...,...,...,...,...,...,...
60,1241,0,UMD_000047_s1,0,0,1,2,0,1.0
61,1257,0,UMD_000252_s1,0,0,2,2,0,1.0
62,1267,0,UMD_000346_s1,0,0,1,2,0,1.0
63,1268,0,UMD_000364_s1,0,0,1,2,0,1.0


In [84]:
PP2A_het_controles_129147457_AJ.to_csv('PP2A_het_controles_129147457_AJ.csv')

In [85]:
temp = pd.read_csv('PP2A_het_controles_129147457_AJ.csv')
temp

,Unnamed: 0,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129129094:A:G_G,chr9:129147457:C:T_T
0,43,0,BIOFIND_000116_s1,0,0,2,1,0,1.0
1,45,0,BIOFIND_000126_s1,0,0,1,1,0,1.0
2,484,0,CORIELL_004515_s1,0,0,2,1,0,1.0
3,485,0,CORIELL_004521_s1,0,0,2,1,0,1.0
4,487,0,CORIELL_004628_s1,0,0,1,1,0,1.0
5,501,0,CORIELL_005008_s1,0,0,1,1,0,1.0
6,517,0,CORIELL_005259_s1,0,0,2,1,0,1.0
7,520,0,CORIELL_005327_s1,0,0,1,1,0,1.0
8,523,0,CORIELL_005425_s1,0,0,2,1,0,1.0
9,532,0,CORIELL_005730_s1,0,0,2,1,0,1.0


In [86]:
PP2A_het_casos_129129094_AJ.to_csv('PP2A_het_casos_129129094_AJ.csv')

In [87]:
temp = pd.read_csv('PP2A_het_casos_129129094_AJ.csv')
temp

,Unnamed: 0,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129129094:A:G_G,chr9:129147457:C:T_T
0,60,0,CORIELL_000035_s1,0,0,2,2,1,0.0
1,67,0,CORIELL_000138_s1,0,0,1,2,1,0.0
2,137,0,CORIELL_001140_s1,0,0,1,2,1,0.0
3,224,0,CORIELL_001997_s1,0,0,1,2,1,0.0
4,301,0,CORIELL_002302_s1,0,0,1,2,1,0.0
5,421,0,CORIELL_003933_s1,0,0,2,2,1,1.0
6,437,0,CORIELL_004167_s1,0,0,2,2,1,0.0
7,718,0,PAGE_001019_s1,0,0,1,2,1,0.0
8,1252,0,UMD_000157_s1,0,0,1,2,1,0.0


In [88]:
PP2A_het_controles_129129094_AJ.to_csv('PP2A_het_controles_129129094_AJ.csv')

In [89]:
temp = pd.read_csv('PP2A_het_controles_129129094_AJ.csv')
temp

,Unnamed: 0,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129129094:A:G_G,chr9:129147457:C:T_T


In [90]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR
head master_key_release3_final.csv

GP2ID,GP2sampleID,manifest_id,phenotype,pheno_for_qc,other_pheno,sex_for_qc,age,age_of_onset,age_at_diagnosis,age_at_death,race_for_qc,family_history_for_qc,region_for_qc,study,pruned,pruned_reason,label,related
SYNAPS-KZ_000001,SYNAPS-KZ_000001_s1,m1,Control,1,Control,2,64.0,,,,Asian,Not Reported,KAZ,SYNAPS-KZ,0,,CAS,
SYNAPS-KZ_000002,SYNAPS-KZ_000002_s1,m1,Control,1,Control,1,53.0,,,,Asian,Not Reported,KAZ,SYNAPS-KZ,0,,CAS,
SYNAPS-KZ_000003,SYNAPS-KZ_000003_s1,m1,Control,1,Control,2,59.0,,,,Asian,Not Reported,KAZ,SYNAPS-KZ,0,,CAS,
SYNAPS-KZ_000004,SYNAPS-KZ_000004_s1,m1,Control,1,Control,1,57.0,,,,Asian,Not Reported,KAZ,SYNAPS-KZ,0,,CAS,
SYNAPS-KZ_000005,SYNAPS-KZ_000005_s1,m1,Control,1,Control,1,59.0,,,,Asian,Not Reported,KAZ,SYNAPS-KZ,0,,CAS,
SYNAPS-KZ_000006,SYNAPS-KZ_000006_s1,m1,Control,1,Control,1,53.0,,,,Asian,Not Reported,KAZ,SYNAPS-KZ,0,,CAS,
SYNAPS-KZ_000007,SYNAPS-KZ_000007_s1,m1,Control,1,Control,1,56.0,,,,Asian,Not Reported,KAZ,SYNAPS-KZ,1,sex_prune,,
SYNAPS-KZ_000008,SY

In [91]:
%%R
setwd('/home/jupyter/PP2A_GP2/')
library(data.table)
covs <- fread("master_key_release3_final.csv", header =T)
covs

                  GP2ID         GP2sampleID manifest_id phenotype pheno_for_qc
    1: SYNAPS-KZ_000001 SYNAPS-KZ_000001_s1          m1   Control            1
    2: SYNAPS-KZ_000002 SYNAPS-KZ_000002_s1          m1   Control            1
    3: SYNAPS-KZ_000003 SYNAPS-KZ_000003_s1          m1   Control            1
    4: SYNAPS-KZ_000004 SYNAPS-KZ_000004_s1          m1   Control            1
    5: SYNAPS-KZ_000005 SYNAPS-KZ_000005_s1          m1   Control            1
   ---                                                                        
16853:      PPMI_000789      PPMI_000789_s1          m2     Other           -9
16854:      PPMI_000781      PPMI_000781_s1          m2     Other           -9
16855:      PPMI_000818      PPMI_000818_s1          m2     Other           -9
16856:      PPMI_000841      PPMI_000841_s1          m2     Other           -9
16857:      PPMI_000808      PPMI_000808_s1          m2     Other           -9
                                  other_pheno sex_fo

data.table 1.14.6 using 2 threads (see ?getDTthreads).  Latest news: r-datatable.com
In addition: Warning message:
In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
  libraries ‘/home/jupyter/packages’, ‘/usr/lib/R/site-library’ contain no packages


In [92]:
%%R
colnames(covs)[2]  <- "IID"  
covs

                  GP2ID                 IID manifest_id phenotype pheno_for_qc
    1: SYNAPS-KZ_000001 SYNAPS-KZ_000001_s1          m1   Control            1
    2: SYNAPS-KZ_000002 SYNAPS-KZ_000002_s1          m1   Control            1
    3: SYNAPS-KZ_000003 SYNAPS-KZ_000003_s1          m1   Control            1
    4: SYNAPS-KZ_000004 SYNAPS-KZ_000004_s1          m1   Control            1
    5: SYNAPS-KZ_000005 SYNAPS-KZ_000005_s1          m1   Control            1
   ---                                                                        
16853:      PPMI_000789      PPMI_000789_s1          m2     Other           -9
16854:      PPMI_000781      PPMI_000781_s1          m2     Other           -9
16855:      PPMI_000818      PPMI_000818_s1          m2     Other           -9
16856:      PPMI_000841      PPMI_000841_s1          m2     Other           -9
16857:      PPMI_000808      PPMI_000808_s1          m2     Other           -9
                                  other_pheno sex_fo

In [93]:
%%R
setwd('/home/jupyter/PP2A_GP2/')
library(data.table)
PP2A_het_casos_129147457_AJ <- fread("PP2A_het_casos_129147457_AJ.csv", header =T)
PP2A_het_casos_129147457_AJ
temp <- merge(covs, PP2A_het_casos_129147457_AJ, by="IID")
head(temp)
write.table(temp, file = "PP2A_het_casos_129147457_AJ_extracted.txt", quote = F, row.names = F, sep = "\t")
PP2A_het_casos_129147457_AJ_extracted <- fread("PP2A_het_casos_129147457_AJ_extracted.txt", header =T)
PP2A_het_casos_129147457_AJ_extracted

                      IID              GP2ID manifest_id phenotype pheno_for_qc
 1:         BCM_000051_s1         BCM_000051          m1        PD            2
 2:         BCM_000158_s1         BCM_000158          m1        PD            2
 3:     BIOFIND_000049_s1     BIOFIND_000049          m1        PD            2
 4:     BIOFIND_000136_s1     BIOFIND_000136          m1        PD            2
 5:     BIOFIND_000205_s1     BIOFIND_000205          m1        PD            2
 6:     CORIELL_000100_s1     CORIELL_000100          m3        PD            2
 7:     CORIELL_000121_s1     CORIELL_000121          m3        PD            2
 8:     CORIELL_000297_s1     CORIELL_000297          m3        PD            2
 9:     CORIELL_000341_s1     CORIELL_000341          m3        PD            2
10:     CORIELL_000527_s1     CORIELL_000527          m3        PD            2
11:     CORIELL_000547_s1     CORIELL_000547          m3        PD            2
12:     CORIELL_000772_s1     CORIELL_00

In [94]:
%%R
setwd('/home/jupyter/PP2A_GP2/')
library(data.table)
PP2A_het_controles_129147457_AJ <- fread("PP2A_het_controles_129147457_AJ.csv", header =T)
PP2A_het_controles_129147457_AJ
temp <- merge(covs, PP2A_het_controles_129147457_AJ, by="IID")
head(temp)
write.table(temp, file = "PTPA_het_129147457_controles_extracted.txt", quote = F, row.names = F, sep = "\t")
PP2A_het_controles_129147457_AJ_extracted <- fread("PP2A_het_controles_129147457_AJ_extracted.txt", header =T)
PP2A_het_controles_129147457_AJ_extracted

                  IID          GP2ID manifest_id phenotype pheno_for_qc
 1: BIOFIND_000116_s1 BIOFIND_000116          m1   Control            1
 2: BIOFIND_000126_s1 BIOFIND_000126          m1   Control            1
 3: CORIELL_004515_s1 CORIELL_004515          m6   Control            1
 4: CORIELL_004521_s1 CORIELL_004521          m6   Control            1
 5: CORIELL_004628_s1 CORIELL_004628          m6   Control            1
 6: CORIELL_005008_s1 CORIELL_005008          m6   Control            1
 7: CORIELL_005259_s1 CORIELL_005259          m6   Control            1
 8: CORIELL_005327_s1 CORIELL_005327          m6   Control            1
 9: CORIELL_005425_s1 CORIELL_005425          m6   Control            1
10: CORIELL_005730_s1 CORIELL_005730          m6   Control            1
11: CORIELL_005747_s1 CORIELL_005747          m6   Control            1
12: CORIELL_005951_s1 CORIELL_005951          m6   Control            1
13: CORIELL_006063_s1 CORIELL_006063          m6   Control      

In [95]:
%%R
setwd('/home/jupyter/PP2A_GP2/')
library(data.table)
PP2A_het_casos_129129094_AJ <- fread("PP2A_het_casos_129129094_AJ.csv", header =T)
PP2A_het_casos_129129094_AJ
temp <- merge(covs, PP2A_het_casos_129129094_AJ, by="IID")
head(temp)
write.table(temp, file = "PP2A_het_casos_129129094_AJ_extracted.txt", quote = F, row.names = F, sep = "\t")
PP2A_het_casos_129129094_AJ_extracted <- fread("PP2A_het_casos_129129094_AJ_extracted.txt", header =T)
PP2A_het_casos_129129094_AJ_extracted

                 IID          GP2ID manifest_id phenotype pheno_for_qc
1: CORIELL_000035_s1 CORIELL_000035          m3        PD            2
2: CORIELL_000138_s1 CORIELL_000138          m3        PD            2
3: CORIELL_001140_s1 CORIELL_001140          m3        PD            2
4: CORIELL_001997_s1 CORIELL_001997          m3        PD            2
5: CORIELL_002302_s1 CORIELL_002302          m3        PD            2
6: CORIELL_003933_s1 CORIELL_003933          m3        PD            2
7: CORIELL_004167_s1 CORIELL_004167          m3        PD            2
8:    PAGE_001019_s1    PAGE_001019          m1        PD            2
9:     UMD_000157_s1     UMD_000157          m1        PD            2
   other_pheno sex_for_qc      age age_of_onset age_at_diagnosis age_at_death
1:          PD          2 60.00000     59.00000               NA           NA
2:          PD          1 56.00000     53.00000               NA           NA
3:          PD          1 52.00000     50.00000         

In [96]:
%%R
setwd('/home/jupyter/PP2A_GP2/')
library(data.table)
PP2A_het_controles_129129094_AJ <- fread("PP2A_het_controles_129129094_AJ.csv", header =T)
PP2A_het_controles_129129094_AJ
temp <- merge(covs, PP2A_het_controles_129129094_AJ, by="IID")
head(temp)
write.table(temp, file = "PP2A_het_controles_129129094_AJ_extracted.txt", quote = F, row.names = F, sep = "\t")
PP2A_het_controles_129129094_AJ_extracted <- fread("PP2A_het_controles_129129094_AJ_extracted.txt", header =T)
PP2A_het_controles_129129094_AJ_extracted

Error in bmerge(i, x, leftcols, rightcols, roll, rollends, nomatch, mult,  : 
  Incompatible join types: x.IID (logical) and i.IID (character)


RInterpreterError: Failed to parse and evaluate line 'setwd(\'/home/jupyter/PP2A_GP2/\')\nlibrary(data.table)\nPP2A_het_controles_129129094_AJ <- fread("PP2A_het_controles_129129094_AJ.csv", header =T)\nPP2A_het_controles_129129094_AJ\ntemp <- merge(covs, PP2A_het_controles_129129094_AJ, by="IID")\nhead(temp)\nwrite.table(temp, file = "PP2A_het_controles_129129094_AJ_extracted.txt", quote = F, row.names = F, sep = "\\t")\nPP2A_het_controles_129129094_AJ_extracted <- fread("PP2A_het_controles_129129094_AJ_extracted.txt", header =T)\nPP2A_het_controles_129129094_AJ_extracted\n'.
R error message: 'Error in bmerge(i, x, leftcols, rightcols, roll, rollends, nomatch, mult,  : \n  Incompatible join types: x.IID (logical) and i.IID (character)'

In [101]:
%%R
setwd('/home/jupyter/PP2A_GP2/')
library(data.table)
PP2A_hom_casos_129147457_AJ <- fread("PP2A_hom_casos_129147457_AJ.csv", header =T)
PP2A_hom_casos_129147457_AJ
temp <- merge(covs, PP2A_hom_casos_129147457_AJ, by="IID")
head(temp)
write.table(temp, file = "PP2A_hom_casos_129147457_AJ_extracted.txt", quote = F, row.names = F, sep = "\t")
PP2A_hom_casos_129147457_AJ_extracted <- fread("PP2A_hom_casos_129147457_AJ_extracted.txt", header =T)
PP2A_hom_casos_129147457_AJ_extracted

                 IID          GP2ID manifest_id phenotype pheno_for_qc
1: BIOFIND_000043_s1 BIOFIND_000043          m1        PD            2
2: CORIELL_001619_s1 CORIELL_001619          m3        PD            2
   other_pheno sex_for_qc age age_of_onset age_at_diagnosis age_at_death
1:          PD          1  82           67               68           NA
2:          PD          2  62           36               36           NA
   race_for_qc family_history_for_qc region_for_qc   study pruned pruned_reason
1:       White                   Yes           USA BIOFIND      0            NA
2:       White                    No           USA CORIELL      0            NA
   label related  V1 FID PAT MAT SEX PHENOTYPE chr9:129129094:A:G_G
1:    AJ      NA  35   0   0   0   1         2                    0
2:    AJ      NA 176   0   0   0   2         2                    0
   chr9:129147457:C:T_T
1:                    2
2:                    2


## Save out results..!

In [ ]:
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR} {WORKSPACE_BUCKET}')